In [25]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
import re
import json
import random
import time
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2 
import io

In [15]:
def get_trail_data():
    # Query SQL trail table
    conn = psycopg2.connect("host=localhost dbname=trailrec user=briangraham")
    cur = conn.cursor()
    sql_query = """
    SELECT * FROM trails;
    """
    trail_data = pd.read_sql_query(sql_query,conn)
    
    #reset indices
    trail_data = trail_data.reset_index()
    # reverse map trail names to indices
    indices = pd.Series(trail_data.index, index=trail_data['trail_id'])
    
    return trail_data,indices

In [16]:
def get_soup(url):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
    page = get(url,headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

In [39]:
def find_id(href):
    return href and re.compile("trailid=").search(href)

In [66]:
trail_data,indices = get_trail_data()
trail_data.head()
#trail_data['url'] = 'https://www.trailforks.com/trails/' + trail_data['trail_id'] + '/'

,index,trail_id,Altitude change,Altitude end,Altitude max,Altitude min,Altitude start,Avg reverse time,Avg time,Bike type,...,description,distance,latitude,longitude,rating,state,votes,Climb Difficulty,Distance flat,trail_num
0,0,1-87-dh,"-1,508 ft",349 ft,"1,857 ft",349 ft,"1,857 ft",00:01:40,00:12:58,"DH, AM",...,Sustained steep and fast DH trail often used a...,2 miles,48.82592,-123.762540,4.55,BC,13,None,None,None
1,1,10-dollar,-191 ft,"2,446 ft","2,636 ft","2,446 ft","2,636 ft",00:15:47,00:09:03,"DH, AM, XC",...,One of the original Pidherny trails.,"2,226 ft",53.98968,-122.846810,3.50,BC,8,None,76 ft,None
2,2,10-km,-694 ft,"4,931 ft","5,625 ft","4,918 ft","5,625 ft",None,00:06:44,DH,...,The 10K trail is a good early season starting ...,"2,840 ft",51.28040,-116.893354,3.20,BC,5,None,None,None
3,3,115th-ave-connector,-46 ft,"1,054 ft",None,"1,054 ft","1,100 ft",None,None,XC,...,Short connector to access the Osoyoos Canal.,"1,417 ft",49.03372,-119.488330,1.00,BC,1,Green,48 ft,None
4,4,147,-0 ft,"2,638 ft","2,638 ft","2,635 ft","2,638 ft",00:07:23,00:01:17,"DH, AM, Fat",...,"Short, technical trail that connects The Swamp...",324 ft,53.87815,-122.828430,4.00,BC,1,Green,64 ft,None


In [52]:
#trail_id_num = []
#for trail in trail_data['url']:
#    soup = get_soup(trail)
#    trail_id_num.append(soup.find(href=find_id).attrs['href'].split('trailid=')[1])

In [53]:
len(trail_id_num)

2802

In [54]:
df_trail_id_info = pd.DataFrame({'trail_data':trail_data['trail_id'],'trail_num':trail_id_num})

In [55]:
df_trail_id_info.head()

,trail_data,trail_num
0,1-87-dh,6592
1,10-dollar,16776
2,10-km,3455
3,115th-ave-connector,135396
4,147,35231


In [56]:
conn = psycopg2.connect("host=localhost dbname=trailrec user=briangraham")
cur = conn.cursor()
cur.execute('ALTER TABLE %s ADD COLUMN %s text' % ('trails', 'trail_num'))
conn.commit()

In [64]:
import psycopg2 
def update_vendor(trail_id, trail_num):
    """ update trail num based on the trail id """
    sql = """ UPDATE trails
                SET trail_num = %s
                WHERE trail_id = %s"""
    conn = None
    updated_rows = 0
    try:
        # read database configuration
        # connect to the PostgreSQL database
        conn = psycopg2.connect("host=localhost dbname=trailrec user=briangraham")
        # create a new cursor
        cur = conn.cursor()
        # execute the UPDATE  statement
        cur.execute(sql, (trail_id, trail_num))
        # get the number of updated rows
        updated_rows = cur.rowcount
        # Commit the changes to the database
        conn.commit()
        # Close communication with the PostgreSQL database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

    return updated_rows

In [69]:
for trail_id, trail_num in zip(,df_trail_id_info['trail_num']):
    update_vendor(trail_id, trail_num)

can't adapt type 'Series'


0

In [70]:
update_vendor(df_trail_id_info['trail_data'][1], df_trail_id_info['trail_num'][1])

AttributeError: 'str' object has no attribute 'str'

In [73]:
df_trail_id_info.to_csv('trail_id_nums.csv')